In [130]:
import yfinance as yf
import pandas as pd


## Masukkan hanya saham LQ-45

In [131]:
# List of ticker symbols
ticker_symbols = [
'ACES.JK',
'ADRO.JK',
'AKRA.JK',
'AMRT.JK',
'ANTM.JK',
'ARTO.JK',
'ASII.JK',
'BBCA.JK',
'BBNI.JK',
'BBRI.JK',
'BBTN.JK',
'BMRI.JK',
'BRIS.JK',
'BRPT.JK',
'BUKA.JK',
'CPIN.JK',
'EMTK.JK',
'ESSA.JK',
'EXCL.JK',
'GGRM.JK',
'GOTO.JK',
'HRUM.JK',
'ICBP.JK',
'INCO.JK',
'INDF.JK',
'INKP.JK',
'INTP.JK',
'ITMG.JK',
'KLBF.JK',
'MAPI.JK',
'MBMA.JK',
'MDKA.JK',
'MEDC.JK',
'MTEL.JK',
'PGAS.JK',
'PGEO.JK',
'PTBA.JK',
'PTMP.JK',
'SIDO.JK',
'SMGR.JK',
'SRTG.JK',
'TLKM.JK',
'TOWR.JK',
'UNTR.JK',
'UNVR.JK'
]

In [132]:
# Function to fetch and transform financial data
def fetch_and_transform_data(ticker_symbol, statement_type):
    try:
        # Fetch financial data based on the specified statement type
        if statement_type == 'CF':
            financial_data = yf.Ticker(ticker_symbol).cashflow
        elif statement_type == 'IS':
            financial_data = yf.Ticker(ticker_symbol).financials
        elif statement_type == 'BS':
            financial_data = yf.Ticker(ticker_symbol).balancesheet

        # Prepare data for transformation
        dfs = []
        available_dates = financial_data.columns
        
        # Iterate over each available date
        for date in available_dates:
            # Extract financial data for the current date
            year = pd.to_datetime(date).year

            data_date = financial_data[date]

            # data_date = financial_data[date]
            
            # Append each account as a row in the DataFrame
            for account, value in data_date.items():
                dfs.append({
                    'symbol': ticker_symbol,
                    'account': account,
                    'type': statement_type,  
                    'year': year,
                    'value': value
                })
        
        # Create DataFrame from the collected data
        return pd.DataFrame(dfs)
    
    except:
        print(f"No data available for {ticker_symbol} ({statement_type})")
        return None

# Fetch and combine data for all ticker symbols
combined_dataframes = []

for ticker_symbol in ticker_symbols:
    # Fetch and transform cash flow data
    cash_flow_data = fetch_and_transform_data(ticker_symbol, 'CF')
    
    # Fetch and transform income statement data
    income_statement_data = fetch_and_transform_data(ticker_symbol, 'IS')

    # Fetch and transform income statement data
    balance_sheet_data = fetch_and_transform_data(ticker_symbol, 'BS')

    # Concatenate dataframes (cash flow and income statement) for the current ticker symbol
    combined_data = pd.concat([cash_flow_data, income_statement_data, balance_sheet_data], ignore_index=True)
    combined_dataframes.append(combined_data)

# Combine all dataframes into a single DataFrame
if combined_dataframes:
    all_data = pd.concat(combined_dataframes, ignore_index=True)

    # Pivot the DataFrame to convert dates into columns
    pivot_data = all_data.pivot_table(index=['symbol', 'account', 'type'], columns='year', values='value').reset_index()

    # Reorder columns to match the desired format
    desired_columns_order = ['symbol', 'account', 'type'] + list(pivot_data.columns[3:])  # Include all columns after 'type'
    
    scrapped_data = pivot_data[desired_columns_order]
    scrapped_data['symbol'] = scrapped_data['symbol'].apply(lambda x: x.split('.')[0])

    # Convert NaN values to empty strings if needed
    scrapped_data = scrapped_data.fillna('')

    filename = 'datasets/lq45_financial_data_raw.csv'
    scrapped_data.to_csv(filename, index=False)

    # Display the final transformed data
    print("Combined financial data:")
    print(scrapped_data)
else:
    print("Unable to retrieve and combine financial data for all ticker symbols.")

Combined financial data:
year symbol                                     account type          2019  \
0      ACES                            Accounts Payable   BS                 
1      ACES                         Accounts Receivable   BS                 
2      ACES                    Accumulated Depreciation   BS                 
3      ACES                  Additional Paid In Capital   BS                 
4      ACES  Allowance For Doubtful Accounts Receivable   BS                 
...     ...                                         ...  ...           ...   
7186   UNVR                         Total Unusual Items   IS -2479000000.0   
7187   UNVR      Total Unusual Items Excluding Goodwill   IS -2479000000.0   
7188   UNVR                      Treasury Shares Number   BS                 
7189   UNVR                             Work In Process   BS                 
7190   UNVR                             Working Capital   BS                 

year             2020             2021